In [399]:
import pandas as pd
# imported world poulation by country and year from https://github.com/87b659ab-0737-4f61-8155-256dd76a8001
df=pd.read_csv("/Users/jaswanth/Downloads/Internet usage by country/population.csv")

# Extracting population data from 2000 to 2023 and renaming columns and ensuring appropriate data types
df_2000to2023= df[(df['Year']>=2000) & (df['Year']<=2023)]
df_2000to2023['Year']=df_2000to2023['Year'].astype('int')
df_2000to2023= df_2000to2023.rename(columns={'Value':'Population'})
df_2000to2023['Population']=df_2000to2023['Population'].astype('int')

/var/folders/0c/1qmcx3ds3vv92czkvs88k9_h0000gn/T/ipykernel_1764/1734395918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000to2023['Year']=df_2000to2023['Year'].astype('int')


In [411]:
# Internet usage
df_internet=pd.read_csv("internet_usage.csv")
years=list(map(lambda x: str(x),range(2000,2024)))
# Handling non numeric values under years by coercion to "NaN"
df_internet[years] = df_internet[years].apply(pd.to_numeric, errors='coerce')
df_internet['t1']=df_internet[years].apply(np.sum,axis=1)
# Drop countries without any data for atleast one year
df_internet = df_internet.drop(df_internet[df_internet['t1'] == 0].index)
df_internet.sort_values('t1')
# reshaping data frame by converting into a flat table
df_internet_flat= df_internet.melt(id_vars=['Country Name','Country Code'],value_vars=years,var_name='Year',value_name='% of Internet_usage')
df_internet_flat['Year'] = df_internet_flat['Year'].astype('int')


ModuleNotFoundError: No module named 'missingno'

In [401]:
# check for country name match in both population and internet usage tables 
Country_code_internet=set(df_internet_flat['Country Name'])
Country_code_population=set(df_2000to2023['Country Name'])
Country_code_internet.issubset(Country_code_population)
df_internet_flat.info()
df_2000to2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country Name         5016 non-null   object 
 1   Country Code         5016 non-null   object 
 2   Year                 5016 non-null   int64  
 3   % of Internet_usage  4589 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 156.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 6360 entries, 40 to 16929
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  6360 non-null   object
 1   Country Code  6360 non-null   object
 2   Year          6360 non-null   int64 
 3   Population    6360 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 248.4+ KB


In [402]:
(df_merged[df_merged['% of Internet_usage'].isna()]['Population']>3000000).sum()

170

In [403]:
# joining the tables and adding new column for number of internet users
df_merged=df_internet_flat.merge(df_2000to2023,on=['Country Name','Country Code','Year'])
df_merged['World population'] = df_merged.groupby('Year')['Population'].transform(sum)
df_merged['% of World population'] =  df_merged['Population'] / df_merged['World population']
# join for including the region of a country
df_region=pd.read_csv('Country_Region.csv',delimiter=';')
df_region= df_region[['ISO-alpha3 Code','Region Name','Sub-region Name']]
df_merged_region = df_merged.merge(df_region,left_on=['Country Code'],right_on='ISO-alpha3 Code' )
df_merged_region = df_merged_region.drop('ISO-alpha3 Code', axis=1)
# join for including the per capita GDP of a country
df_income = pd.read_csv('income_group.csv')
df_income.head()
df_final= df_merged_region.merge(df_income,on=['Country Code'])
df_final = df_final.iloc[:,:]
df_final=df_final.drop(['Region', 'SpecialNotes','TableName', 'Unnamed: 5'],axis=1)
df_GDP = pd.read_csv('GDP_per capita.csv')
df_GDP= df_GDP[['Country Code',*years]]
df_GDP= df_GDP.melt(id_vars=['Country Code'],value_vars=years,var_name='Year',value_name='GDP per capita')
df_GDP['Year']=df_GDP['Year'].astype(int)
df_final.info()
df_final= df_final.merge(df_GDP,on=['Country Code','Year'])
df_final.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4992 entries, 0 to 4991
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country Name           4992 non-null   object 
 1   Country Code           4992 non-null   object 
 2   Year                   4992 non-null   int64  
 3   % of Internet_usage    4587 non-null   float64
 4   Population             4992 non-null   int64  
 5   World population       4992 non-null   int64  
 6   % of World population  4992 non-null   float64
 7   Region Name            4992 non-null   object 
 8   Sub-region Name        4992 non-null   object 
 9   IncomeGroup            4968 non-null   object 
dtypes: float64(2), int64(3), object(5)
memory usage: 390.1+ KB


Index(['Country Name', 'Country Code', 'Year', '% of Internet_usage',
       'Population', 'World population', '% of World population',
       'Region Name', 'Sub-region Name', 'IncomeGroup', 'GDP per capita'],
      dtype='object')

In [404]:
missing_values = df_final['% of Internet_usage'].isna().sum()
print(f"Number of missing values in '% of Internet_usage': {missing_values}")

Number of missing values in '% of Internet_usage': 405


In [405]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Separate rows with missing values in '% of Internet_usage'
train_data = df_final[df_final['% of Internet_usage'].notna()]
test_data = df_final[df_final['% of Internet_usage'].isna()]

# Features and target variable
X_train = train_data[['Year', 'Population', 'Region Name', 'Sub-region Name', 'IncomeGroup']]  # Features
y_train = train_data['% of Internet_usage']  # Target

# For the rows with missing values in '% of Internet_usage', we'll only use the features
X_test = test_data[['Year', 'Population', 'Region Name', 'Sub-region Name', 'IncomeGroup']]

# Numeric columns (that may need standardization)
numeric_cols = ['Year', 'Population']

# Categorical columns (that need encoding)
categorical_cols = ['Region Name', 'Sub-region Name', 'IncomeGroup']

# Create a column transformer to handle both numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Apply StandardScaler to numeric columns
        ('cat', OneHotEncoder(), categorical_cols)  # Apply OneHotEncoder to categorical columns
    ])

# Initialize the Linear Regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),  # First, apply preprocessing to the data
    ('regressor', LinearRegression())  # Then, apply Linear Regression
])

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions for the test data (where internet usage is missing)
y_pred = model.predict(X_test)

# Replace the missing values in '% of Internet_usage' with the predicted values
df_final.loc[df_final['% of Internet_usage'].isna(), '% of Internet_usage'] = y_pred

# (Optional) Evaluate the model on training data
y_train_pred = model.predict(X_train)
mse = mean_squared_error(y_train, y_train_pred)
print(f'Mean Squared Error on Training Data: {mse}')


Mean Squared Error on Training Data: 156.63544739876636


In [406]:
# Find the minimum positive value in '% of Internet_usage'
min_positive_value = df_final[df_final['% of Internet_usage'] > 0]['% of Internet_usage'].min()
max_positive_value = df_internet_flat['% of Internet_usage'].max()

# Impute negative and zero values with the minimum positive value
df_final['% of Internet_usage'] = df_final['% of Internet_usage'].apply(lambda x: min_positive_value if x <= 0 else x)
df_final['% of Internet_usage'] = df_final['% of Internet_usage'].apply(lambda x: max_positive_value if x >=100 else x)

# Check the result
print(df_final['% of Internet_usage'].describe())


count    4992.000000
mean       39.625167
std        31.626983
min         0.000289
25%         8.119155
50%        35.076094
75%        68.687040
max       100.000000
Name: % of Internet_usage, dtype: float64


In [407]:
df_final

,Country Name,Country Code,Year,% of Internet_usage,Population,World population,% of World population,Region Name,Sub-region Name,IncomeGroup,GDP per capita
0,Afghanistan,AFG,2000,0.000289,20130327,6115617907,0.003292,Asia,Southern Asia,Low income,174.930991
1,Afghanistan,AFG,2001,0.004723,20284307,6198523633,0.003272,Asia,Southern Asia,Low income,138.706822
2,Afghanistan,AFG,2002,0.004561,21378117,6280690978,0.003404,Asia,Southern Asia,Low income,178.954088
3,Afghanistan,AFG,2003,0.087891,22733049,6362582991,0.003573,Asia,Southern Asia,Low income,198.871116
4,Afghanistan,AFG,2004,0.105809,23560654,6445292478,0.003655,Asia,Southern Asia,Low income,221.763654
...,...,...,...,...,...,...,...,...,...,...,...
4987,Zimbabwe,ZWE,2019,26.588300,15271368,7726797600,0.001976,Africa,Sub-Saharan Africa,Lower middle income,1684.027904
4988,Zimbabwe,ZWE,2020,29.298600,15526888,7805953886,0.001989,Africa,Sub-Saharan Africa,Lower middle income,1730.413489
4989,Zimbabwe,ZWE,2021,32.461600,15797210,7871000477,0.002007,Africa,Sub-Saharan Africa,Lower middle income,1724.387731
4990,Zimbabwe,ZWE,2022,32.561500,16069056,7940003773,0.002024,Africa,Sub-Saharan Africa,Lower middle income,2040.552459


In [408]:
df_final.to_csv("internet_usage & population.csv")


